In [ ]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [ ]:
l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT","L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [ ]:
"""
CREATE TABLE SUPPLIER (
    S_SUPPKEY        SERIAL,
    S_NAME            CHAR(25),
    S_ADDRESS        VARCHAR(40),
    S_NATIONKEY        INTEGER NOT NULL, -- references N_NATIONKEY
    S_PHONE            CHAR(15),
    S_ACCTBAL        DECIMAL,
    S_COMMENT        VARCHAR(101)
);
"""

s_columnnames = ["S_SUPPKEY", "S_NAME", "S_ADDRESS", "S_NATIONKEY", "S_PHONE", "S_ACCTBAL", "S_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [ ]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../tpch-pgsql-master/data/load/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

In [ ]:
df_filter_1 = lineitem[(lineitem_1.l_shipdate >= pd.Timestamp('1996-01-01 00:00:00')) & (lineitem_1.l_shipdate < pd.Timestamp('1996-04-01 00:00:00'))]
df_filter_1 = df_filter_1[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_filter_1['l_extendedprice1l_discount'] = df_filter_1.l_extendedprice * ( 1 - df_filter_1.l_discount )
df_group_1 = df_filter_1 \
    .groupby(['l_suppkey']) \
    .agg(
        suml_extendedprice1l_discount=("l_extendedprice1l_discount", "sum"),
    )
df_group_1 = df_group_1[['suml_extendedprice1l_discount']]
df_aggr_1 = pd.DataFrame()
df_aggr_1['maxsuml_extendedprice1l_discount'] = [(suml_extendedprice1l_discount).max()]
df_aggr_1 = df_aggr_1[['maxsuml_extendedprice1l_discount']]
dollar_0 = df_aggr_1

df_filter_1 = supplier[['s_suppkey', 's_name', 's_address', 's_nationkey', 's_phone', 's_acctbal', 's_comment']]
df_filter_2 = lineitem[(lineitem.l_shipdate >= pd.Timestamp('1996-01-01 00:00:00')) & (lineitem.l_shipdate < pd.Timestamp('1996-04-01 00:00:00'))]
df_filter_2 = df_filter_2[['l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment']]
df_filter_2['l_extendedprice1l_discount'] = df_filter_2.l_extendedprice * ( 1 - df_filter_2.l_discount )
df_group_1 = df_filter_2 \
    .groupby(['l_suppkey']) \
    .agg(
        suml_extendedprice1l_discount=("l_extendedprice1l_discount", "sum"),
    )
df_group_1 = df_group_1[df_group_1.suml_extendedprice1l_discount == dollar_0]
df_group_1 = df_group_1[['suml_extendedprice1l_discount']]
df_group_1 = df_group_1.rename_axis(['l_suppkey']).reset_index()
df_rename_1 = pd.DataFrame()
df_rename_1['total_revenue'] = df_group_1['suml_extendedprice1l_discount']
df_rename_1['supplier_no'] = df_group_1['l_suppkey']
df_sort_1 = df_rename_1.sort_values(by=['supplier_no'], ascending=[True])
df_sort_1 = df_sort_1[['total_revenue', 'supplier_no']]
df_merge_1 = df_filter_1.merge(df_sort_1, left_on="s_suppkey", right_on="supplier_no")
df_merge_1 = df_merge_1[['s_suppkey', 's_name', 's_address', 's_phone', 'total_revenue']]
df_limit_1 = df_merge_1[['s_suppkey', 's_name', 's_address', 's_phone', 'total_revenue']]
result = df_limit_1.head(1)
return result
